In [21]:
import math
import numpy as np
import timeit
import matplotlib.pyplot as plt
from ipywidgets import * # ez kell az interaktiv plottolashoz
from mpl_toolkits import mplot3d # a harom dimenzios plottolashoz kell
from IPython.display import display # a save buttonhoz

data=np.loadtxt('circ_test.txt') # itt töltöm a circ_test.txt file-t

e = data[:,0]
a = data[:,1]
tau = data[:,5]
ink = data[:,2]
om = data[:,3]
w = data[:,4] # pályaelemek beolvasása a fileból
   
length = len(a) # a félnagytengely tömbbe lévő elemek száma, ez minden pályaelemre ugyan ennyi lenne. 

for i in range (length): # itt használom ki, hogy az előbb megnézte a program, hogy hány elemű a tömb.
    w[i] = math.radians(w[i])
    om[i] = math.radians(om[i])
    ink[i] = math.radians(ink[i]) # radiánba átváltom a fokban megadott mennyiségeket.
    
num_pairs = 1

In [22]:
a_res = []
e_res = []
ink_res = []
tau_res = []
om_res = []
w_res = []

for i in range (2*num_pairs):
    a_res.append(a[i])
    e_res.append(e[i])
    ink_res.append(ink[i])
    tau_res.append(tau[i])
    om_res.append(om[i])
    w_res.append(w[i])
        
a_res = np.array(a_res)
e_res = np.array(e_res)
ink_res = np.array(ink_res)
om_res = np.array(om_res)
tau_res = np.array(tau_res)
w_res = np.array(w_res) # ezekbe a tömbökbe tárolom a beolvasott adatokat, ennek az értékei a pályamódosításkor változni fognak.

a_ = [] # ezek a ismét a beolvasott pályaelemek, ezeket a tömböket végig békén hagyom
e_ = []
ink_ = []
tau_ = []
om_ = []
w_ = []

# ezeket a listakat azert hozom letre, mert a palyaelemek_res listaknak az elemeit modositani fogom a sebessegvaltoztatasokkal
# ezek segitsegevel majd vizsgalhato, hogy mennyit valtoztak a palyaelemek a modositastol
for i in range (2*num_pairs):
    a_.append(a[i])
    e_.append(e[i])
    ink_.append(ink[i])
    tau_.append(tau[i])
    om_.append(om[i])
    w_.append(w[i])

In [23]:
# szukseges fuggvenyek definialasa:

v = np.linspace(0,2*np.pi,1000)
v_len = len(v)
k = 0.01720209895
mu = k**2

def save(b):
    np.savetxt("modositott_palyaelemek.txt", b) # a megváltoztatott pályaelemeket ebbe mentem

def parameter (a, e): # a dolgozatomban 14. egyenletben definiált p paraméter
    p = a*(1-e**2)
    return p

def focal_equation (p, e, v): # 13. egyenlet
    return p/(1+e*np.cos(v)) 

def eta (r, v): # a 18. egyenletben szerepel
    return r*np.sin(v)

def xsi (r, v): # a 18. egyenletben szerepel
    return r*np.cos(v)

def P_x (w, om, i): # a 18. egyenletben szerepel
    return np.cos(w)*np.cos(om)-np.sin(w)*np.sin(om)*np.cos(i)

def P_y (w, om, i): # a 18. egyenletben szerepel
    return np.cos(w)*np.sin(om)+np.sin(w)*np.cos(om)*np.cos(i)

def P_z (w, om, i): # a 18. egyenletben szerepel
    return np.sin(w)*np.sin(i)

def Q_x (w, om, i): # a 18. egyenletben szerepel
    return -np.sin(w)*np.cos(om)-np.cos(w)*np.sin(om)*np.cos(i)

def Q_y (w, om, i): # a 18. egyenletben szerepel
    return -np.sin(w)*np.sin(om)+np.cos(w)*np.cos(om)*np.cos(i)

def Q_z (w, om,i): # a 18. egyenletben szerepel
    return np.cos(w)*np.sin(i)

def fn_xyz (w, om, ink, r, v): # a 18. egyenlet megoldása
    matrix = [[P_x(w,om,ink), Q_x(w,om,ink)], [P_y(w,om,ink), Q_y(w,om,ink)], [P_z(w,om,ink), Q_z(w,om,ink)]]
    vector = [xsi(r,v), eta(r,v)]
    xyz = np.matmul(matrix, vector)
    return xyz

def fn_E(E,e,M): # iterációhoz szükséges, hogy a Kepler egyenletet megoldhassuk
    fn_E = E - e * np.sin(E) - M
    return fn_E

def fn_der_E(E,e): # az fn_E függvény deriváltja, az iterációhoz szükséges
    return 1-e*np.cos(E)

def E_0(M,e): # iterálás
    return M + ((e*np.sin(M))/(1-np.sin(M+e)+np.sin(M)))

def difference_iteration(E0, e, M): # az iterálást elvégző függvény
    return E0 - fn_E(E0, e, M) / fn_der_E(E0, e)

def true_anomaly_from_E(E, e): # 22. egyenlet
    tol = 1e-7
    if abs(E - np.pi) < tol:
        return np.pi
    if abs(E) < tol:
        return 0
    if abs(E - 3*np.pi/2) < tol:
        return 3*np.pi/2
    else:
        return 2 * math.atan(math.sqrt((1+e)/(1-e))*math.tan(E/2))
    
def eccentric_anomaly_from_v(v,e): # 22. egyenlet
    tol = 1e-7
    if abs(v) < tol:
        return 0
    else:
        return 2 * math.atan(math.sqrt((1-e)/(1+e))*math.tan(v/2))

def xsi_deriv (v, e, a): #20. egyenlet
    p = a*(1-e**2)
    return -math.sqrt(mu/p)*np.sin(v)

def eta_deriv (v, e, a): #21. egyenlet
    p = a*(1-e**2)
    return math.sqrt(mu/p)*(np.cos(v)+e)

def fn_velocity_xyz (a, e, w, om, ink, v): # 19. egyenlet
    matrix = [[P_x(w,om,ink), Q_x(w,om,ink)], [P_y(w,om,ink), Q_y(w,om,ink)], [P_z(w,om,ink), Q_z(w,om,ink)]]
    vector = [xsi_deriv(v, e, a), eta_deriv(v, e, a)]
    v_xyz = np.matmul(matrix, vector)
    return v_xyz

def fn_c(xyz, v_xyz): # 10. egyenlet
    return np.cross(xyz, v_xyz)

def fn_c_abs(c): # az impulzus-momentum vektor abszolút értéke
    return math.sqrt(c[0]**2 + c[1]**2 + c[2]**2)

def fn_v_abs(v_xyz): # a sebesség vektor abszolút értéke
    return (v_xyz[0])**2 + (v_xyz[1])**2 + (v_xyz[2])**2

def fn_h (v_xyz_square, r): # 11. egyenlet
    return 0.5 * (v_xyz_square) - mu / r

def fn_lambda(r, v_xyz, xyz, c): # 12. egyenlet
    return -mu / r * xyz + np.cross(v_xyz, c)

def fn_lambda_abs(lambda_vec): # lambda abszolút értéke
    return math.sqrt(lambda_vec[0]**2 + lambda_vec[1]**2 + lambda_vec[2]**2)

def fn_inclination(c, c_abs): # 27. egyenlet
    return math.asin(math.sqrt(c[0]**2+c[1]**2)/c_abs)

def fn_longitude_ascending_node(c,c_abs): # 28. egyenlet
    om = math.atan2(math.sqrt(c[0]**2+c[1]**2),c_abs)
    if om < 0:
        return om + 2*np.pi
    else:
        return om
    
def fn_semi_major_axis(h): # 24. egyenlet
    return -mu/(2*h) 

def fn_eccentricity(h, c_abs):  #25. egyenlet
    return math.sqrt(1 + 2 * h * (c_abs / mu)**2)

def fn_argument_pericenter(c_abs, c, lambda_vec): # 26. egyenlet
    w = math.atan2((c_abs*lambda_vec[2]), ((c[0]*lambda_vec[1])-(c[1]*lambda_vec[0])))
    if w < 0:
        return w + 2*np.pi
    else:
        return w 

def fn_time_pericenter(JD, e, E, a): # 29. egyenlet
    return JD-(E - e*np.sin(E))/math.sqrt(mu/a**3)
    
def fn_true_anomaly(e, M):
    
    tol = 1e-7
    E_null_JD_original = 1
    E_JD_original = []
    v_JD_original = []
    
    for i in range (num_pairs):
        while True:
            E_next_JD_original = difference_iteration(E_null_JD_original, e, M)
            if abs(E_next_JD_original - E_null_JD_original) > tol:
                E_null_JD_original = E_next_JD_original
            else:
                E_JD_original.append(E_null_JD_original)
                break

    for i in range (num_pairs):
        if E_JD_original[i] < 0:
            E_JD_original[i] = E_JD_original[i] + 2 * np.pi
            
    for i in range (num_pairs):
        v_JD_original.append(true_anomaly_from_E((E_JD_original[i]), e))
        
    if v_JD_original[i] < 0:
        v_JD_original[i] = v_JD_original[i] + 2 * np.pi
    
    return v_JD_original # iteráció segítségével ez a függvény adja meg a valódi anomáliát a középanomália és az excentricitás alapján.

In [24]:
def plot_slider_with_velocity_JD(vx, vy, vz, JD_boost):
   
    # az objektumparok egyike, ezeken nem valtoztatok:
    
    M_JD_original = []
    v_JD_original = []

    xyz_JD_original = []
    xyz_original = []
    r_JD_original = []
    r_original = [] # a szükséges listákat itt hozom létre

    
    for i in range (num_pairs): 
        M_JD_original.append(math.sqrt(mu) * a_res[2*i]**(-3/2) * (JD_boost - tau_res[2*i]))
    M_JD_original = np.array(M_JD_original) #kozepanomalia kiszámítása a sliderrel megadott JD-on
    
    for i in range(num_pairs):
        v_JD_original.append(fn_true_anomaly(e_res[2*i], M_JD_original[i]))
    v_JD_original = np.array(v_JD_original) # valódi anomália 

    for i in range (num_pairs):
        r_JD_original.append(focal_equation(parameter(a_res[2*i],e_res[2*i]), e_res[2*i], v_JD_original[i])) 
        xyz_JD_original.append(fn_xyz(w_res[2*i],  om_res[2*i],  ink_res[2*i], r_JD_original[i], v_JD_original[i])) # az objektum x-y-z koordinátája a beállított julián dátumon
        r_original.append(focal_equation(parameter(a_res[2*i], e_res[2*i]), e_res[2*i],v)) # teljes pálya
        xyz_original.append(fn_xyz(w_res[2*i],  om_res[2*i],  ink_res[2*i], r_original[i], v)) # pálya x-y-z koordinátái
   
    xyz_original = np.array(xyz_original)
    xyz_JD_original = np.array(xyz_JD_original)

    
    # xyz,v_xyz az eredeti palyakon a paratlan szamu aszteroidara:
    # ameddig nem valtoztatom a sebessegeket:
    if vx == vy == vz == 0:
        
        M_JD = []
        v_JD = []
        
        for i in range (num_pairs): 
            M_JD.append(math.sqrt(mu) * a_res[2*i+1]**(-3/2) * (JD_boost - tau_res[2*i+1]))
        M_JD = np.array(M_JD) #kozepanomalia a sliderrel megadott JD-on

        for i in range(num_pairs):
            v_JD.append(fn_true_anomaly(e_res[2*i+1], M_JD[i]))
        v_JD = np.array(v_JD)
       
        xyz = []
        v_xyz = []
        r = []
        r_orbit = []
        xyz_orbit = []    
        for i in range (num_pairs):
            r.append(focal_equation(parameter(a_res[2*i+1], e_res[2*i+1]), e_res[2*i+1], v_JD[i]))
            xyz.append(fn_xyz(w_res[2*i+1],  om_res[2*i+1],  ink_res[2*i+1], r[i], v_JD[i]))
            v_xyz.append(fn_velocity_xyz(a_res[2*i+1], e_res[2*i+1], w_res[2*i+1],  om_res[2*i+1],  ink_res[2*i+1], v_JD[i]))
            # az objektumok pálya menti sebességét a v_xyz tömbben tárolom.
            r_orbit.append(focal_equation(parameter(a_res[2*i+1],e_res[2*i+1]),e_res[2*i+1],v))
            xyz_orbit.append(fn_xyz(w_res[2*i+1],  om_res[2*i+1],  ink_res[2*i+1], r_orbit[i], v))
                
        
        xyz = np.array(xyz)
        xyz_orbit = np.array(xyz_orbit)
        v_xyz = np.array(v_xyz)
        
        # a fenti módon ismét kiszámítható az objektum pályája
        
        for n in range (num_pairs): # objektumpáronkénti ábrák készítése:
            fig = plt.figure()
            fig.set_size_inches(10, 10)
            ax = fig.add_subplot(111, projection='3d')
            ax.plot(xyz_original[n,0], xyz_original[n,1], xyz_original[n,2], color = 'green', linewidth='3', label = '1. égitest pályája')
            ax.plot(xyz_orbit[n,0], xyz_orbit[n,1], xyz_orbit[n,2], color = 'orange', linewidth='3', label = '2. égitest palyja')

            ax.scatter(0,0,0, color = 'black', marker = 'o', label = 'Nap')
            ax.scatter(xyz_JD_original[n,0], xyz_JD_original[n,1], xyz_JD_original[n,2], s = 50, color = 'blue', marker = 'o', label = '1. égitest')
            ax.scatter(xyz[n,0], xyz[n,1], xyz[n,2], s = 50, color = 'red', marker = 'o', label = '2. égitest')
            ax.set_xlabel('X tengely')
            ax.set_ylabel('Y tengely')
            ax.set_zlabel('Z tengely')
            ax.legend(loc = 'upper right')  
            
    else:  # ha a pályamenti sebességeken módosítunk, akkor fut le az else ág.
    
        M_JD_dv = []
        E_JD_dv = []
        v_JD_dv = []
        
        for i in range (num_pairs): 
            M_JD_dv.append(math.sqrt(mu) * a_res[2*i+1]**(-3/2) * (JD_boost - tau_res[2*i+1]))
        M_JD_dv = np.array(M_JD_dv) #kozepanomalia a sliderrel megadott JD-on

        for i in range(num_pairs):
            v_JD_dv = fn_true_anomaly(e_res[2*i+1], M_JD_dv[i])
        v_JD_dv = np.array(v_JD_dv)
        
    # elso modositastol ez fog lefutni, ha JD-ket allitgatom v_JD_dv[i] fog valtozni   
        
        r_dv = []
        xyz_dv = []
        v_xyz_dv = []
        r_orbit_dv = []
        xyz_orbit_dv = []
        
        for i in range (num_pairs):
            r_dv.append(focal_equation(parameter(a_res[2*i+1], e_res[2*i+1]), e_res[2*i+1], v_JD_dv[i]))
            xyz_dv.append(fn_xyz(w_res[2*i+1],  om_res[2*i+1], ink_res[2*i+1], r_dv[i], v_JD_dv[i]))
            v_xyz_dv.append(fn_velocity_xyz(a_res[2*i+1], e_res[2*i+1], w_res[2*i+1], om_res[2*i+1], ink_res[2*i+1], v_JD_dv[i]))
            # a teljes modositott palya:
            r_orbit_dv.append(focal_equation(parameter(a_res[2*i+1], e_res[2*i+1]), e_res[2*i+1], v))
            xyz_orbit_dv.append(fn_xyz(w_res[2*i+1],  om_res[2*i+1], ink_res[2*i+1], r_orbit_dv[i], v))
        
        v_xyz_dv = np.array(v_xyz_dv)
        xyz_dv = np.array(xyz_dv)
        xyz_orbit_dv = np.array(xyz_orbit_dv)    
        r_dv = np.array(r_dv)
        #a pálya meghatározása a már ismertetett módon.
        
        # a sebessegeket itt adom hozza a v_xyz_dv tombhoz, az integralok mar ezekkel fognak dolgozni
        for i in range (num_pairs):
            v_xyz_dv[i,0] = v_xyz_dv[i,0]+vx
            v_xyz_dv[i,1] = v_xyz_dv[i,1]+vy
            v_xyz_dv[i,2] = v_xyz_dv[i,2]+vz
            
        c = []
        c_abs = []
        lambda_abs = []
        lambda_vec = []
        h_energy = []
        
        
        # az impulzus-momentum integrál:
        for i in range (num_pairs):
            c.append(fn_c(xyz_dv[i], v_xyz_dv[i]))
        c = np.array(c)
      
        # a Laplace-integrál:
        for i in range (num_pairs):
            lambda_vec.append(fn_lambda(r_dv[i], v_xyz_dv[i], xyz_dv[i], c[i]))
        lambda_vec = np.array(lambda_vec) 
        
        # num_pairs x 3as tomb, elso indexeben num_pairs szamu lambda_x stb.
        
        v_xyz_square = []
        for i in range (num_pairs):
            v_xyz_square.append(fn_v_abs(v_xyz_dv[i]))
        v_xyz_square = np.array(v_xyz_square) # ez egy num_pairs elemu tomb, a sebessegek abszolutertekenek negyzeteit tartalmazza
        # az energia kiszamitasakor ezt hasznalom
        
        for i in range(num_pairs):
            lambda_abs.append(fn_lambda_abs(lambda_vec[i]))
            h_energy.append(fn_h(v_xyz_square[i], r_dv[i]))
            c_abs.append(fn_c_abs(c[i]))
            
        lambda_abs = np.array(lambda_abs)
        h_energy = np.array(h_energy)
        c_abs = np.array(c_abs)
        
        # itt számítom ki majd kapják meg a palyaelemek_res tömbök a sebességmódosítás utáni értéket:
        for i in range (num_pairs):
            a_res[2*i+1] = (fn_semi_major_axis(h_energy[i])) 
            ink_res[2*i+1] = (fn_inclination(c[i], c_abs[i]))
            om_res[2*i+1] = (fn_longitude_ascending_node(c[i], c_abs[i]))
            e_res[2*i+1] = (fn_eccentricity(h_energy[i], c_abs[i]))
            w_res[2*i+1] = (fn_argument_pericenter(c_abs[i], c[i], lambda_vec[i]))
            
        for i in range(num_pairs):
            if e_res[2*i+1] >= 1:
                print('Asteroid number', i+1 ,'is no longer on an elliptical orbit!') # az elliptikus pálya feltétének teljesülése.
            
        for i in range(num_pairs):
            E_JD_dv.append(eccentric_anomaly_from_v(v_JD_dv[i],e_res[2*i+1]))
        E_JD_dv = np.array(E_JD_dv) # az excentrikus anomália, erre a pericentrum átmenet időpontjának megadásakot van szükség
        
        for i in range (num_pairs):
            tau_res[2*i+1] = (fn_time_pericenter(JD_boost, e_res[2*i+1], E_JD_dv[i], a_res[2*i+1]))
        
        # sebesseg modositast kovetoen a palya:
        
        r_next_JD = []
        xyz_next_JD = []
        v_xyz_next_JD = []
        r_orbit_next = []
        xyz_orbit_next = []
        
        # az itt kiszamitott palyaknal nem szamolok ujra, a sebessegmodositas utani v_xyz alapjan kapott 
        # palyaelemekbol ujra valodi anomaliat, mert a dv sebessegvaltoztatas pillanatszeruen tortenik,
        # addig a valodi anomalia nem fog megvaltozni (ezert marad itt is a v_JD_dv).
        
        for i in range (num_pairs):
            r_next_JD.append(focal_equation(parameter(a_res[2*i+1], e_res[2*i+1]), e_res[2*i+1],v_JD_dv[i]))
            xyz_next_JD.append(fn_xyz(w_res[2*i+1],  om_res[2*i+1], ink_res[2*i+1], r_next_JD[i], v_JD_dv[i]))
            v_xyz_next_JD.append(fn_velocity_xyz(a_res[2*i+1], e_res[2*i+1], w_res[2*i+1], om_res[2*i+1], ink_res[2*i+1], v_JD_dv[i]))
            # a teljes modositott palya:
            r_orbit_next.append(focal_equation(parameter(a_res[2*i+1], e_res[2*i+1]), e_res[2*i+1],v))
            xyz_orbit_next.append(fn_xyz(w_res[2*i+1],  om_res[2*i+1], ink_res[2*i+1], r_orbit_next[i], v))
     
        v_xyz_next_JD = np.array(v_xyz_next_JD)
        xyz_orbit_next = np.array(xyz_orbit_next)
        xyz_next_JD= np.array(xyz_next_JD)
        
        # a pálya kiszámítása és az ábrázolás a már ismertetett módon.
        
        for n in range (num_pairs):
            fig = plt.figure()
            fig.set_size_inches(10, 10)
            ax = fig.add_subplot(111, projection='3d')
            ax.plot(xyz_original[n,0], xyz_original[n,1], xyz_original[n,2], color = 'green', linewidth='3', label = '1. égitest pályája (valtozatlan)')
            ax.plot(xyz_orbit_next[n,0], xyz_orbit_next[n,1], xyz_orbit_next[n,2], color = 'orange', linewidth='3', label = '2. égitest pályája (valtoztatott)')

            ax.scatter(0,0,0, color = 'black', marker = 'o', label = 'Nap')
            ax.scatter(xyz_JD_original[n,0], xyz_JD_original[n,1], xyz_JD_original[n,2], s = 50, color = 'blue', marker = 'o', label = '1. égitest (valtozatlan)')
            ax.scatter(xyz_next_JD[n,0], xyz_next_JD[n,1], xyz_next_JD[n,2], s = 50, color = 'red', marker = 'o', label = '2. égitest (valtoztatott)')
            ax.set_xlabel('X tengely')
            ax.set_ylabel('Y tengely')
            ax.set_zlabel('Z tengely')
            ax.legend(loc = 'upper right')
        
  # a paronkent vett objektumok palyaelemeinek különbsége -> milyen közel vannak
        d_a = []
        d_a_original = []
        d_e = []
        d_e_original = []
        d_ink = []
        d_ink_original = []
        d_w = []
        d_w_original = []
        d_om = []
        d_om_original = []
        d_tau = []
        d_tau_original = []
        for i in range (num_pairs):
            d_a.append(abs(a_res[2*i+1]-a_res[2*i]))
            d_a_original.append(abs(a_[2*i+1]-a_[2*i]))
            d_e.append(abs(e_res[2*i+1]-e_res[2*i]))
            d_e_original.append(abs(e_[2*i+1]-e_[2*i]))
            d_ink.append(abs(ink_res[2*i+1]-ink_res[2*i]))
            d_ink_original.append(abs(ink_[2*i+1]-ink_[2*i]))
            d_w.append(abs(w_res[2*i+1]-w_res[2*i]))
            d_w_original.append(abs(w_[2*i+1]-w_[2*i]))
            d_om.append(abs(om_res[2*i+1]-om_res[2*i]))
            d_om_original.append(abs(om_[2*i+1]-om_[2*i]))
            d_tau.append(abs(tau_res[2*i+1]-tau_res[2*i]))
            d_tau_original.append(abs(tau_[2*i+1]-tau_[2*i]))
            
        print(#'A felnagytengelyek kulonbsege a sebessegvaltoztatas elott: \n', d_a_original, '\n',
              'A felnagytengelyek kulonbsege a sebessegvaltoztatas utan: \n', d_a, '\n',
              #'Az excentricitasok kulonbsege a sebessegvaltoztatas elott: \n', d_e_original, '\n',
              'Az excentricitasok kulonbsege a sebessegvaltoztatas utan: \n', d_e, '\n',
              #'Az inklinaciok kulonbsege a sebessegvaltoztatas elott: \n', d_ink_original, '\n',
              'Az inklinaciok kulonbsege a sebessegvaltoztatas utan: \n', d_ink, '\n',
              #'A pericentrum argumentumok kulonbsege a sebessegvaltoztatas elott: \n', d_w_original, '\n',
              'A pericentrum argumentumok kulonbsege a sebessegvaltoztatas utan: \n',  d_w, '\n',
              #'A felszallo csomo hosszak kulonbsege a sebessegvaltoztatas elott: \n', d_om_original, '\n',
              'A felszallo csomo hosszak kulonbsege a sebessegvaltoztatas utan: \n', d_om, '\n',
              #'A pericentrum atmenetek idopontjanak kulonbsege a sebessegvaltoztatas elott: \n', d_tau_original, '\n',
              'A pericentrum atmenetek idopontjanak kulonbsege a sebessegvaltoztatas utan: \n', d_tau, '\n')
        
        save((a_res,e_res,ink_res,tau_res,om_res,w_res))

In [25]:
def objects_position_after_dv(JD): # a már ismertetett módon ábrázolja ez a függvény az objektumpárok pályáját
  # a transzfer után hívom meg, hogy látható lehessen utána milyen a pálya
  # a pillanatnyi távolságot kiírja a függvény, így vizsgálható, hogy mikor milyen messze vannak az objektumok.
    orb_elements=np.loadtxt('modositott_palyaelemek.txt')

    e_from_file = orb_elements[1,:]
    a_from_file = orb_elements[0,:]
    tau_from_file = orb_elements[3,:]
    ink_from_file = orb_elements[2,:]
    om_from_file = orb_elements[4,:]
    w_from_file = orb_elements[5,:]
    
    M_JD = []
    v_JD = []
    
    for i in range (2*num_pairs): 
        M_JD.append(math.sqrt(mu) * a_from_file[i]**(-3/2) * (JD - tau_from_file[i]))
    M_JD = np.array(M_JD)
    
    for i in range(2*num_pairs):
        v_JD.append(fn_true_anomaly(e_from_file[i], M_JD[i]))
    v_JD = np.array(v_JD)
    
    xyz_JD = []
    r_JD = []
    r = []
    xyz = []

    for i in range (2*num_pairs):
        r_JD.append(focal_equation(parameter(a_from_file[i],e_from_file[i]), e_from_file[i], v_JD[i]))
        xyz_JD.append(fn_xyz(w_from_file[i],  om_from_file[i],  ink_from_file[i], r_JD[i], v_JD[i]))
        r.append(focal_equation(parameter(a_from_file[i], e_from_file[i]), e_from_file[i],v))
        xyz.append(fn_xyz(w_from_file[i],  om_from_file[i],  ink_from_file[i], r[i], v))
   
    xyz = np.array(xyz)
    xyz_JD = np.array(xyz_JD)
    
    dist_x = []
    dist_y = []
    dist_z = []
    distance = []
    distance_km = []
    
    for i in range (0, 2*num_pairs, 2):
        dist_x.append(abs(xyz_JD[i,0]-xyz_JD[i+1,0]))
        dist_y.append(abs(xyz_JD[i,1]-xyz_JD[i+1,1]))
        dist_z.append(abs(xyz_JD[i,2]-xyz_JD[i+1,2]))
        
    for i in range(num_pairs):
        distance .append(math.sqrt(dist_x[i]**2+dist_y[i]**2+dist_z[i]**2))
        
    for i in range (num_pairs):    
        distance_km.append(distance[i] * 1.5e8)
        
    print('Az objektumok egymástól mért aktuális távolsága kiométerben:','\n', distance_km, '\n',
         'Az objektumok egymástól mért aktuális távolsága csillagászati egységben:','\n', distance)
    
    for n in range (0, 2*num_pairs, 2):
        fig = plt.figure()
        fig.set_size_inches(10, 10)
        ax = fig.add_subplot(111, projection='3d')
        ax.plot(xyz[n,0], xyz[n,1], xyz[n,2], color = 'green', linewidth='3', label = '1. aszteroida palyaja (valtozatlan)')
        ax.plot(xyz[n+1,0], xyz[n+1,1], xyz[n+1,2], color = 'orange', linewidth='3', label = '2. aszteroida palyaja (valtoztatott)')

        ax.scatter(0,0,0, color = 'black', marker = 'o', label = 'Nap')
        ax.scatter(xyz_JD[n,0], xyz_JD[n,1], xyz_JD[n,2], color = 'blue', marker = 'o', s = 50, label = '1. aszeroida (valtozatlan)')
        ax.scatter(xyz_JD[n+1,0], xyz_JD[n+1,1], xyz_JD[n+1,2], color = 'red', marker = 'o', s = 50, label = '2. aszeroida (valtoztatott)')
        ax.set_xlabel('X tengely')
        ax.set_ylabel('Y tengely')
        ax.set_zlabel('Z tengely')
        ax.legend(loc = 'upper right')

In [26]:
# fuggvenyek a Hohmann transfer sebessegadasokhoz:

def fn_v1(r1, r2): # a 30. egyenlet
    return math.sqrt(mu/r1)*(math.sqrt(2*r2/(r1+r2))-1)

def fn_v2(r1, r2): # 31. egyenlet
    return math.sqrt(mu/r2)*(1-math.sqrt(2*r1/(r1+r2)))

def fn_v_total(v1, v2): # a teljes delta v
    return v1 + v2 

def fn_half_period(r1, r2): # fél periódusnyi idő, ennyi elteltével kell a második delta v.
    return np.pi*math.sqrt(((r1+r2)/2)**3/mu)

t_half = fn_half_period(a_[1],a_[0])

v1 = fn_v1(a_[1],a_[0])
v2 = fn_v2(a_[1],a_[0])
v_tot = fn_v_total(v1, v2)

print('Az első sebességmódosítás értéke: ', v1, 'CSE/nap', '\n',
      'A második sebességmódosítás értéke: ', v2, 'CSE/nap', '\n',
      'A teljes sebességmódosítás nagysága: ', v_tot, 'CSE/nap \n',
      'A transzfer végrehajtásához szükséges idő: ', t_half, 'nap')

Az első sebességmódosítás értéke:  0.0016916422786281678 CSE/nap 
 A második sebességmódosítás értéke:  0.0015226517769338273 CSE/nap 
 A teljes sebességmódosítás nagysága:  0.003214294055561995 CSE/nap 
 A transzfer végrehajtásához szükséges idő:  258.2999064499475 nap


In [27]:
# fuggvenyek a Bi-elliptikus transfer sebessegadasokhoz:

rb = 5

def fn_a1(r1, rb):
    return (r1+rb)/2

def fn_a2(r2, rb):
    return (r2+rb)/2

def fn_v1_b(r1, r2, a1): # 33. egyenlet
    return math.sqrt(2*mu/r1 - mu/a1) - math.sqrt(mu/r1) 

def fn_v2_b(rb, a1, a2): # 34. egyenlet
    return math.sqrt(2*mu/rb - mu/a2) - math.sqrt(2*mu/rb - mu/a1)

def fn_v3_b(r2, a2): # 35. egyenlet
    return math.sqrt(2*mu/r2 - mu/a2) - math.sqrt(mu/r2)

def fn_v_total_b(v1, v2, v3):
    return abs(v1) + abs(v2) + abs(v3)
    
def fn_t1_b(a1):
    return np.pi*math.sqrt(a1**3/mu)

def fn_t2_b(a2): # periódusidők fele, mikor a következő delta v adható.
    return np.pi*math.sqrt(a2**3/mu)
    
def fn_time_transfer_b(t1, t2):
    return t1 + t2

a1 = fn_a1(a_[1], rb)
a2 = fn_a2(a_[0], rb)

t1_b = fn_t1_b(a1)
t2_b = fn_t2_b(a2)

if rb == a_[0]:
    t_tot_b = t2_b
else:
    t_tot_b = fn_time_transfer_b(t1_b, t2_b)

v1_b = fn_v1_b(a_[1], a_[0], a1)
v2_b = fn_v2_b(rb, a1, a2)
v3_b = fn_v3_b(a_[0], a2)
v_tot_b = fn_v_total_b(v1_b, v2_b, v3_b)

# a_[0] itt az r2, a_[1] pedig az r1

print('A bi-elliptikus transzfer adatai: \n',
      'Az első sebességmódosítás értéke: ', v1_b, 'CSE/nap \n',
      'A második sebességmódosítás értéke: ', v2_b, 'CSE/nap \n',
      'A harmadik sebességmódosítás értéke: ', v3_b, 'CSE/nap \n',
      'A teljes sebességmódosítás nagysága: ', v_tot_b, 'CSE/nap \n',
      'A transzfer végrehajtásához szükséges idő: ', t_tot_b, 'nap \n')

print('A Hohmann-transzfer adatai: \n',
      'Az első sebességmódosítás értéke: ', v1, 'CSE/nap \n',
      'A második sebességmódosítás értéke: ', v2, 'CSE/nap \n',
      'A teljes sebességmódosítás nagysága: ', v_tot, 'CSE/nap \n',
      'A transzfer végrehajtásához szükséges idő: ', t_half, 'nap \n')


if v_tot < v_tot_b:
    print('A megadott elrendezésben a Hohmann-transzfer igényelt kevesebb energia befektetést!')
else:
    print('A megadott elrendezésben a bi-elliptikus transzfer igényelt kevesebb energia befektetést!')


A bi-elliptikus transzfer adatai: 
 Az első sebességmódosítás értéke:  0.00500571530105403 CSE/nap 
 A második sebességmódosítás értéke:  0.0008114617333706917 CSE/nap 
 A harmadik sebességmódosítás értéke:  0.0033269414396973045 CSE/nap 
 A teljes sebességmódosítás nagysága:  0.009144118474122026 CSE/nap 
 A transzfer végrehajtásához szükséges idő:  2023.9315137648305 nap 

A Hohmann-transzfer adatai: 
 Az első sebességmódosítás értéke:  0.0016916422786281678 CSE/nap 
 A második sebességmódosítás értéke:  0.0015226517769338273 CSE/nap 
 A teljes sebességmódosítás nagysága:  0.003214294055561995 CSE/nap 
 A transzfer végrehajtásához szükséges idő:  258.2999064499475 nap 

A megadott elrendezésben a Hohmann-transzfer igényelt kevesebb energia befektetést!


In [28]:
a_res = []
e_res = []
ink_res = []
tau_res = []
om_res = []
w_res = []

# azert kell, hogy a kovetkezo futtataskor a palyaelemek_res tombokbe ne a mar modositgatott ertekek legyenek
# hanem ujra az erdetik
# igy hogy ez itt van a fenti appendeles a palyaelemek_res-be nem is szukseges
for i in range (2*num_pairs):
    a_res.append(a[i])
    e_res.append(e[i])
    ink_res.append(ink[i])
    tau_res.append(tau[i])
    om_res.append(om[i])
    w_res.append(w[i])

interact(plot_slider_with_velocity_JD,
         vx = BoundedFloatText(value = 0, min = -1, max = 1,  step = v1),
         vy = BoundedFloatText(value = 0, min = 0, max = v1,  step = v1),
         vz = BoundedFloatText(value = 0, min = -1, max = 1,  step = 0.01),
         JD_boost = FloatSlider(value = 0, min = 0, max = 1e7,  step = 10));

interactive(children=(BoundedFloatText(value=0.0, description='vx', max=1.0, min=-1.0, step=0.0016916422786281…

In [29]:
interact(plot_slider_with_velocity_JD,
         vx = BoundedFloatText(value = 0, min = -1, max = 1,  step = 1),
         vy = BoundedFloatText(value = 0, min = -v2, max = 0,  step = v2),
         vz = BoundedFloatText(value = 0, min = -1, max = 1,  step = 1),
         JD_boost = FloatSlider(value = t_half, min = 0, max = 1e7,  step = 10));

interactive(children=(BoundedFloatText(value=0.0, description='vx', max=1.0, min=-1.0, step=1.0), BoundedFloat…

In [30]:
interact(objects_position_after_dv,
         JD = FloatSlider(value = 0, min = 0, max = 5000,  step = 20))

interactive(children=(FloatSlider(value=0.0, description='JD', max=5000.0, step=20.0), Output()), _dom_classes…

<function __main__.objects_position_after_dv(JD)>

In [31]:
a_res = []
e_res = []
ink_res = []
tau_res = []
om_res = []
w_res = []

# azert kell, hogy a kovetkezo futtataskor a palyaelemek_res tombokbe ne a mar modositgatott ertekek legyenek
# hanem ujra az erdetik
# igy hogy ez itt van a fenti appendeles a palyaelemek_res-be nem is szukseges
for i in range (2*num_pairs):
    a_res.append(a[i])
    e_res.append(e[i])
    ink_res.append(ink[i])
    tau_res.append(tau[i])
    om_res.append(om[i])
    w_res.append(w[i])

interact(plot_slider_with_velocity_JD,
         vx = BoundedFloatText(value = 0, min = -10, max = 10,  step = 1),
         vy = BoundedFloatText(value = 0, min = 0, max = 1000,  step = v1_b),
         vz = BoundedFloatText(value = 0, min = -10, max = 10,  step = 1),
         JD_boost = FloatSlider(value = 0, min = 0, max = 1e7, step = 10));

interactive(children=(BoundedFloatText(value=0.0, description='vx', max=10.0, min=-10.0, step=1.0), BoundedFlo…

In [32]:
interact(plot_slider_with_velocity_JD,
         vx = BoundedFloatText(value = 0, min = -1000, max = 1000,  step = 1),
         vy = BoundedFloatText(value = 0, min = -v2_b, max = 1,  step = v2_b),
         vz = BoundedFloatText(value = 0, min = -1000, max = 1000,  step = 1),
         JD_boost = FloatSlider(value = t1_b, min = 0, max = 1e7,  step = 10));

interactive(children=(BoundedFloatText(value=0.0, description='vx', max=1000.0, min=-1000.0, step=1.0), Bounde…

In [33]:
interact(plot_slider_with_velocity_JD,
         vx = BoundedFloatText(value = 0, min = -1000, max = 1000,  step = 1),
         vy = BoundedFloatText(value = 0, min = -v3_b, max = 0,  step = v3_b),
         vz = BoundedFloatText(value = 0, min = -1000, max = 1000,  step = 1),
         JD_boost = FloatSlider(value = t_tot_b, min = 0, max = 1e7,  step = 10));

interactive(children=(BoundedFloatText(value=0.0, description='vx', max=1000.0, min=-1000.0, step=1.0), Bounde…